# VAE encoder

> a VAE module for percpetion.

In [ ]:
#| default_exp models.vae

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore import *
from fastcore.utils import *

In [ ]:
#| export
"""
Variational encoder model, used as a visual model
for our model of the world.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    """VAE encoder for 32×32 RGB images"""
    def __init__(self, img_channels, latent_size):
        super().__init__()

        self.conv1 = nn.Conv2d(img_channels,  32, 4, stride=2, padding=1)   # 32→16
        self.conv2 = nn.Conv2d(32,  64, 4, stride=2, padding=1)             # 16→8
        self.conv3 = nn.Conv2d(64, 128, 4, stride=2, padding=1)             # 8→4
        self.conv4 = nn.Conv2d(128, 256, 4, stride=2, padding=1)            # 4→2

        self.fc_mu        = nn.Linear(256 * 2 * 2, latent_size)
        self.fc_logsigma  = nn.Linear(256 * 2 * 2, latent_size)

    def forward(self, x):
        x = F.relu(self.conv1(x))  
        x = F.relu(self.conv2(x))  
        x = F.relu(self.conv3(x))  
        x = F.relu(self.conv4(x))  
        print(x.shape)
        x = x.view(x.size(0), -1)  # flatten

        mu = self.fc_mu(x)
        logsigma = self.fc_logsigma(x)
        return mu, logsigma





In [ ]:
#| export
class Decoder(nn.Module):
    """VAE decoder for 32×32 RGB images"""
    def __init__(self, img_channels, latent_size):
        super().__init__()

        self.fc1 = nn.Linear(latent_size, 256 * 2 * 2)

        self.deconv1 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1)  # 2→4
        self.deconv2 = nn.ConvTranspose2d(128,  64, 4, stride=2, padding=1)  # 4→8
        self.deconv3 = nn.ConvTranspose2d(64,   32, 4, stride=2, padding=1)  # 8→16
        self.deconv4 = nn.ConvTranspose2d(32, img_channels, 4, stride=2, padding=1) # 16→32

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 256, 2, 2)

        x = F.relu(self.deconv1(x))
        x = F.relu(self.deconv2(x))
        x = F.relu(self.deconv3(x))

        reconstruction = torch.sigmoid(self.deconv4(x))
        return reconstruction



In [ ]:
#| hide
dec = Decoder(3, 32)
z = torch.randn(16, 32)
recon = dec(z)
recon.shape

torch.Size([16, 3, 32, 32])

In [ ]:
# #| export
# class VAE(nn.Module):
#     """ Variational Autoencoder """
#     def __init__(self, img_channels, latent_size):
#         super(VAE, self).__init__()
#         self.encoder = Encoder(img_channels, latent_size)
#         self.decoder = Decoder(img_channels, latent_size)

#     def forward(self, x): # pylint: disable=arguments-differ
#         mu, logsigma = self.encoder(x)
#         sigma = logsigma.exp()
#         eps = torch.randn_like(sigma)
#         z = eps.mul(sigma).add_(mu)

#         recon_x = self.decoder(z)
#         return recon_x, mu, logsigma

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===============================================================
#                         VAE MODEL
# ===============================================================

class VAE(nn.Module):
    def __init__(self, in_channels=3, latent_dim=128):
        super().__init__()

        self.latent_dim = latent_dim

        # Encoder hidden channels
        hidden_dims = [32, 64, 128, 256]   # 4 blocks only

        # -----------------------
        #        Encoder
        # -----------------------
        modules = []
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, h_dim, kernel_size=3,
                              stride=2, padding=1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU()
                )
            )
            in_channels = h_dim

        # 32x32 → 16 → 8 → 4 → 2  ⟹ final shape = 256×2×2
        self.encoder = nn.Sequential(*modules)

        flattened_dim = hidden_dims[-1] * 2 * 2  # 256*4 = 1024

        self.fc_mu  = nn.Linear(flattened_dim, latent_dim)
        self.fc_var = nn.Linear(flattened_dim, latent_dim)

        # -----------------------
        #        Decoder
        # -----------------------
        self.decoder_input = nn.Linear(latent_dim, flattened_dim)

        hidden_dims = hidden_dims[::-1]   # reverse: [256, 128, 64, 32]

        modules = []
        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride=2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU()
                )
            )

        self.decoder = nn.Sequential(*modules)

        # Final layer to get back to 32x32 resolution
        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(hidden_dims[-1],
                               hidden_dims[-1],
                               kernel_size=3,
                               stride=2,
                               padding=1,
                               output_padding=1),
            nn.BatchNorm2d(hidden_dims[-1]),
            nn.LeakyReLU(),
            nn.Conv2d(hidden_dims[-1], 3, kernel_size=3, padding=1),
            nn.Tanh()  # output in [-1, 1]
        )

    # ===============================================================
    #                       ENCODER
    # ===============================================================
    def encode(self, x):
        h = self.encoder(x)
        h = torch.flatten(h, start_dim=1)
        mu = self.fc_mu(h)
        log_var = self.fc_var(h)
        return mu, log_var

    # ===============================================================
    #                   REPARAMETERIZATION
    # ===============================================================
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    # ===============================================================
    #                       DECODER
    # ===============================================================
    def decode(self, z):
        h = self.decoder_input(z)
        h = h.view(-1, 256, 2, 2)
        h = self.decoder(h)
        h = self.final_layer(h)
        return h

    # ===============================================================
    #                       FORWARD PASS
    # ===============================================================
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        recon = self.decode(z)
        return recon, x, mu, log_var


In [ ]:
#| hide
model = VAE(3, 128)
x = torch.randn(16, 3, 32, 32)
u, lo = model.encode(x)
x_hat = model.decode(u)
x_hat.shape


torch.Size([16, 3, 32, 32])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()